

## "Информационно-развлекательный чат-бот"



## Данный бот позволяет следующее:

### получить курс основных валют ЦБ России на сегодня;

### получить сводку погоды по городам мира;

### получить справочную информацию из википедии;

### просто позадавать любые вопросы в развлекательных целях ("болталка").

In [1]:
pip install feedparser

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Импорт необходимых библиотек
import jsonlines
import pandas as pd

import telebot
import feedparser
import time
from telebot import types
import requests
from datetime import datetime

import telebot, wikipedia, re

import os
import datetime

import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

from sklearn.pipeline import make_pipeline

### Блок "Болталка"

In [3]:
#Escaping out HTML character
!pip install jsonlines
 

В качестве исходных данных был взят датасет из https://huggingface.co/

Это датасет русских диалогов собранных с Telegram чатов. Количество диалогов - 2 миллиона.
https://huggingface.co/datasets/Den4ikAI/russian_dialogues/blob/main/dataset.jsonl

In [4]:
#Соединяем датасет в список, удобный для дальнейшей работы
# Исключаем "пустые" диалоги
dialogs = []
with open('../chat-bot/dataset.jsonl', 'rb') as f:
    for item in jsonlines.Reader(f):
    #for item in f:
        question = item.get("question")
        answer = item.get("answer")
        if question is not None and answer is not None:
            dialogs.append((question, answer))

In [5]:
dff = pd.DataFrame(dialogs, columns=['question', 'answer'])
dff

,question,answer
0,как дела?,там хорошо
1,"вы кефир пачему не кушаете, не любите?",я ряженку лучше люблю.
2,если в расходную накладную забить дури и выкур...,особенно когда придет комиссия проверять докум...
3,покажись в шапке,ды щаз приветик
4,давай не будем об этом,давай поговорим о чем-нибудь другом
...,...,...
2476078,а ваша гармонь отчего поет?,"нет, просто наслаждаюсь пением."
2476079,а кто или что мешает вам быть истинно счастливым?,да. это просто. надо радоваться каждому мгнове...
2476080,как сделать визу в нигерию без справки с работы?,"безработных туда не пускают, значит никак"
2476081,есть универсальный способ проверки состояния м...,да выжать и отпустить сцепление на заведеной н...


In [6]:
rep = dff[dff.question == 'как дела?']
rep

,question,answer
0,как дела?,там хорошо
5624,как дела?,да походу тут все такие собрались
7590,как дела?,"не помню, но ему содрали кожу до мяса блять."
12547,как дела?,ну мне только черный нужен
14480,как дела?,как обычно
...,...,...
2451655,как дела?,"все замечательно, ты как?"
2456111,как дела?,обсуждаем скоротечность времени
2458946,как дела?,"нормально. я с башкирии, если что. мы все тут ..."
2466883,как дела?,хорошоты как? что делаешь?


Видно, что на один и тот-же вопрос есть несколько ответов.

In [8]:
# Рандомно сократим 80% датафрейма
dff = dff.sample(frac=0.2)
dff

,question,answer
191349,кто лучше брюнетки или блондинки?,"блондинки посимпатичнее и миловиднее, ну это м..."
119608,"представляю, какая ржака",все хорошо. спасибо
1191271,нашел че вспомнить.,зоммбиии против крипера погналиии!!!
802915,чем заняться в свободное время?,"спортом, изучением иностранных языков, чтением..."
1870021,станет ли для вас сюрпризом неожиданное призна...,"конечно, приятным, ведь настоящая любовь она в..."
...,...,...
1010246,у тебя мозги всегда включены?,"душа то живое существо, имеющеетело и дхание."
694177,так откроем же банку пива и оптимистично пошле...,о хороший план я в деле
2159526,в какие игры можно поиграть с микроволновкой?,ни в какие. это не игрушка. сломается где носк...
1787459,вы выделяетесь из толпы? какова сила вашей хар...,выделяюсь. это-то многих и раздражает а мне о ...


#### Векторизация текста

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer()
vectorizer.fit(dff.question)

TfidfVectorizer()

In [11]:
# Записываем в матрицу, сколько раз каждое слово встречалось в тексте
matrix_big = vectorizer.transform(dff.question)
matrix_big.shape

(495217, 193794)

Возникла проблема: в базе почти 2,5 млн текстовых «вопросов», в которых содержится 421781 различных слов. Если превратить все вопросы в векторы, получится матрица2476083*421781. Работать с такой компьютеру будет тяжело, поэтому необходимо провести
сокращение размерности.

Попробуем использовать алгоритм PCA (метод главных компонент) 

Сократим размерность до 300 измерений

In [12]:
from sklearn.decomposition import TruncatedSVD

In [13]:
svd = TruncatedSVD(n_components=300)

In [14]:
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)
print('Размер малой матрицы', matrix_small.shape)

Размер малой матрицы (495217, 300)


In [15]:
print('Степень сжатия (потеря информации)', svd.explained_variance_ratio_.sum())

Степень сжатия (потеря информации) 0.24620589563288292


#### Поиск ближайших соседей

Здесь я буду отображать вопросы в Евклидово пространство не очень высокой размерности. Поиск соседей в нём можно осуществлять довольно быстро. Я воспользуюсь уже готовым алгоритмом поиска соседей BallTree. 

Буду использовать модель, которая выбирала бы одного из k ближайших соседей, причём чем ближе сосед, тем выше вероятность его выбора. Ибо брать всегда одного самого близкого соседа — скучно, но не завязываться на сходство совсем — опасно.

In [15]:
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

In [16]:
# Используем функцию softmax для определения «вероятности» выбора ответа.
def softmax(x):
    proba = np.exp(-x)
    return proba/sum(proba)

In [17]:
class NeighborSampler(BaseEstimator):
     def __init__(self, k=5, temperature=1):
            self.k = k
            self.temperature = temperature
     
     def fit(self, X, y):
            self.tree_ = BallTree(X)
            self.y_ = np.array(y)
      
     def predict(self, X, random_state = None):
        distances, indices = self.tree_.query(X, return_distance=True, k = self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distance*self.temperature)))
        return self.y_[result]

Фразы, которые будет вводить пользователь, надо пропускать через все три алгоритма — векторизатор, метод главных компонент, и алгоритм выбора ответа. Чтобы писать меньше кода, можно связать их в единую цепочку (pipeline), применяющую алгоритмы последовательно.

#### Соединяем векторизацию, сокращение размерностей и поиск соседей


In [18]:
from sklearn.pipeline import make_pipeline

In [19]:
ns = NeighborSampler()
ns.fit(matrix_small, dff.answer)
pipe = make_pipeline(vectorizer, svd, ns)

#### Тестируем пайплайн

In [25]:
pipe.predict(['как делишки?'])

array(['меня тоже постоянно яростно будят'], dtype=object)

In [21]:
pipe.predict(['где ты живешь?'])

array(['нет, но я догадываюсь о твоей внешности'], dtype=object)

In [26]:
pipe.predict(['какая погода  у вас?'])

array(['главней всего погода в доме. спокойная, без бурь и катастроф.'],
      dtype=object)

In [23]:
pipe.predict(['чего бы купить?'])

array(['я видела как ты на ихнем трындел'], dtype=object)

In [24]:
pipe.predict(['чего бы посмотреть?'])

array(['от спермотоксикоза ну как вариант'], dtype=object)

In [27]:
pipe.predict(['чего бы поесть?'])

array(['так ты ж в статусе его дрюкаешь'], dtype=object)

### Блок "Википедия"

In [28]:

# Устанавливаем русский язык в Wikipedia
wikipedia.set_lang("ru")
# Чистим текст статьи в Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:1000]
        # Разделяем по точкам
        wikimas=wikitext.split('.')
        # Отбрасываем всЕ после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
                    # Если в строке осталось больше трех символов, 
                    # добавляем ее к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        wikitext2=re.sub('\n', ' ', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'К сожалению, в википедии нет информации об этом'


#### Тестируем запросы

In [29]:
getwiki("слон")

'Слоно́вые, или слоны́ , — семейство класса млекопитающих из отряда хоботных. В настоящее время к этому семейству относятся 3 ныне живущих вида. Африканские саванные слоны — наиболее крупные наземные млекопитающие.'

In [30]:
getwiki("пшеница")

'Пшени́ца  — род травянистых, в основном однолетних, растений семейства Злаки, или Мятликовые , ведущая зерновая культура во многих странах. Получаемая из зёрен пшеницы мука используется при выпекании хлеба, изготовлении макаронных и кондитерских изделий. Пшеница также используется как кормовая культура, входит в некоторые рецепты приготовления пива и водки, а также виски.'

In [31]:
getwiki("костапавв")

'К сожалению, в википедии нет информации об этом'

### Блок "Погода в городе"

In [32]:
def get_weather(town):
    try:
        response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={town}&lang=ru&units=metric&appid=46e7d06c1e690398a33534855c353586")
        data = response.json()
        
        city = data["name"]
        cur_temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        pressure = data["main"]["pressure"]
        wind = data["wind"]["speed"]

        sunrise_timestamp = datetime.datetime.fromtimestamp(data["sys"]["sunrise"])
        sunset_timestamp = datetime.datetime.fromtimestamp(data["sys"]["sunset"])

        # продолжительность дня
        length_of_the_day = datetime.datetime.fromtimestamp(data["sys"]["sunset"]) -       datetime.datetime.fromtimestamp(data["sys"]["sunrise"])

        
         # Возвращаем текстовую строку
        return (f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}\n"\
                 f"Погода в городе: {city}\nТемпература: {cur_temp}°C\n"\
                 f"Влажность: {humidity}%\nДавление: {math.ceil(pressure/1.333)} мм.рт.ст\nВетер: {wind} м/с \n"
                 f"Восход солнца: {sunrise_timestamp}\nЗакат солнца: {sunset_timestamp}\nПродолжительность дня: {length_of_the_day}\n"
                 f"Хорошего дня!"
                )
    
        #return(data)
    except Exception as e:
        return "Проверьте название города!"

#### Тестируем запросы

In [33]:
print(get_weather("Псков"))

2024-03-18 11:14
Погода в городе: Псков
Температура: -0.05°C
Влажность: 93%
Давление: 765 мм.рт.ст
Ветер: 6 м/с 
Восход солнца: 2024-03-18 07:13:31
Закат солнца: 2024-03-18 19:15:42
Продолжительность дня: 12:02:11
Хорошего дня!


In [35]:
print(get_weather("Санкт-Петербург"))

2024-03-18 11:14
Погода в городе: Санкт-Петербург
Температура: -3.05°C
Влажность: 80%
Давление: 767 мм.рт.ст
Ветер: 6 м/с 
Восход солнца: 2024-03-18 07:05:55
Закат солнца: 2024-03-18 19:07:58
Продолжительность дня: 12:02:03
Хорошего дня!


In [34]:
print(get_weather("Пскив"))

Проверьте название города!


In [43]:
bot = telebot.TeleBot('6866368317:AAFIfjoU5Si-3T9jSh1rFcQcaJc0cYmiKgA')
#bot = TeleBot("")
 
 
keyboard1 = types.InlineKeyboardMarkup()
callback_button1 = types.InlineKeyboardButton(text="Курс валют на сегодня ЦБ России", callback_data="1")
callback_button2 = types.InlineKeyboardButton(text="Погода в городе", callback_data="2")
callback_button3 = types.InlineKeyboardButton(text="Википедия", callback_data="3")
callback_button4 = types.InlineKeyboardButton(text="Просто 'болталка'", callback_data="4")
keyboard1.add(callback_button1, callback_button2, callback_button3, callback_button4)
 
 
keyboard2 = types.InlineKeyboardMarkup()
callback_button1 = types.InlineKeyboardButton(text="Назад", callback_data="*")
keyboard2.add(callback_button1)
 
 
@bot.message_handler(commands=['start'])
def cmd_start(message: types.Message) -> None:
    bot.send_message(
        message.chat.id,
        "Выберите вопрос который вас интересует:",
        reply_markup=keyboard1
    )
 
@bot.callback_query_handler(func=lambda callback: True)
def callback_inline(callback: types.CallbackQuery) -> None:
    if callback.data == "*":
        bot.send_message(
            callback.message.chat.id,
            "Выберите вопрос который вас интересует:",
            reply_markup=keyboard1
        )
    
    
    elif callback.data == '1':
            req = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
            response = req.json()
            usd_CBRF = response['Valute']['USD']['Value']
            eur_CBRF = response['Valute']['EUR']['Value']
            bot.send_message(callback.message.chat.id, f"На сегодня курс доллара равен: {usd_CBRF} руб.")
            bot.send_message(callback.message.chat.id, f"На сегодня курс евро равен: {eur_CBRF} руб.")    
            bot.send_message(
            callback.message.chat.id,
            "Можно вернуться в основное меню",
            reply_markup=keyboard2
                            )
    
    elif callback.data == '2':
        bot.send_message(
            callback.message.chat.id,
            "Выберите город, который Вас интересует:")
        
        @bot.message_handler(content_types=["text"])
        def handle_text(message):
            bot.send_message(message.chat.id, get_weather(message.text))

            bot.send_message(
            callback.message.chat.id,
            "Можно вернуться в основное меню",
            reply_markup=keyboard2)
    
    elif callback.data == '3':
        bot.send_message(
            callback.message.chat.id,
            "Введите вопрос, который Вас интересует:")
        @bot.message_handler(content_types=["text"])
        def handl_text(message):
            bot.send_message(message.chat.id, getwiki(message.text))    
            bot.send_message(
            callback.message.chat.id,
            "Можно вернуться в основное меню",
            reply_markup=keyboard2)
    
    elif callback.data == "4":
        bot.send_message(
            callback.message.chat.id,
            "Давайте просто поболтаем:")
        @bot.message_handler(content_types=["text"])
        def hand_text(message):
            bot.send_message(message.chat.id, pipe.predict([message.text.lower()])[0])      
        
        bot.send_message(
            callback.message.chat.id,
            "Можно вернуться в основное меню",
            reply_markup=keyboard2)
# Запускаем бота            
bot.polling(none_stop=True, interval=0)        

#### Вспомогательная информация

In [36]:
def get_data():
    req = requests.get("https://yobit.net/api/3/ticker/btc_usd")
    response = req.json()
    sell_price = response["btc_usd"]["sell"]
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M')}\nSell BTC price: {sell_price}")

In [37]:
req = requests.get("https://yobit.net/api/3/ticker/btc_usd")
response = req.json()
sell_price = response["btc_usd"]["sell"]
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M')}\nSell BTC price: {sell_price}")

AttributeError: module 'datetime' has no attribute 'now'

In [38]:
req.json()

{'btc_usd': {'high': 69999.99999999,
  'low': 66901,
  'avg': 68450.49999999,
  'vol': 23499.46583073,
  'vol_cur': 0.33877886,
  'last': 69999.9,
  'buy': 69450.1,
  'sell': 69999.9,
  'updated': 1710749545}}

In [39]:
req = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
response = req.json()

In [40]:
req.json()

{'Date': '2024-03-16T11:30:00+03:00',
 'PreviousDate': '2024-03-15T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2024/03/15/daily_json.js',
 'Timestamp': '2024-03-18T11:00:00+03:00',
 'Valute': {'AUD': {'ID': 'R01010',
   'NumCode': '036',
   'CharCode': 'AUD',
   'Nominal': 1,
   'Name': 'Австралийский доллар',
   'Value': 60.331,
   'Previous': 60.6263},
  'AZN': {'ID': 'R01020A',
   'NumCode': '944',
   'CharCode': 'AZN',
   'Nominal': 1,
   'Name': 'Азербайджанский манат',
   'Value': 54.0412,
   'Previous': 53.9035},
  'GBP': {'ID': 'R01035',
   'NumCode': '826',
   'CharCode': 'GBP',
   'Nominal': 1,
   'Name': 'Фунт стерлингов Соединенного королевства',
   'Value': 117.1067,
   'Previous': 117.2665},
  'AMD': {'ID': 'R01060',
   'NumCode': '051',
   'CharCode': 'AMD',
   'Nominal': 100,
   'Name': 'Армянских драмов',
   'Value': 22.8175,
   'Previous': 22.7582},
  'BYN': {'ID': 'R01090B',
   'NumCode': '933',
   'CharCode': 'BYN',
   'Nominal': 1,
   'Name': '

In [41]:
response["Valute"]['USD']['Value']

91.87

In [42]:
print(f"{response['Date']}\nUSD cours: {response['Valute']['USD']['Value']}")
print(f"EUR cours: {response['Valute']['EUR']['Value']}")

2024-03-16T11:30:00+03:00
USD cours: 91.87
EUR cours: 99.9718
